In [3]:
(set! *print-length* 10)
(set! *warn-on-reflection* true)

(require 
      '[clojure.string :as s])

nil

In [5]:
(defn parse [data]
    (map read-string (map str data)))

(defn day-input []
                (slurp "../input/day16.txt"))

(day-input)

"59718730609456731351293131043954182702121108074562978243742884161871544398977055503320958653307507508966449714414337735187580549358362555889812919496045724040642138706110661041990885362374435198119936583163910712480088609327792784217885605021161016819501165393890652993818130542242768441596060007838133531024988331598293657823801146846652173678159937295632636340994166521987674402071483406418370292035144241585262551324299766286455164775266890428904814988362921594953203336562273760946178800473700853809323954113201123479775212494228741821718730597221148998454224256326346654873824296052279974200167736410629219931381311353792034748731880630444730593"

In [92]:
(defn pattern ^long [n m]
  ;;  (println (str n ":" m))
 (let [base [0, 1, 0, -1]
       idx  (base (mod (quot  (inc m) n) 4))]
  idx))
 


(map #(pattern 2 %) (range 0 10))


(0 1 1 0 0 -1 -1 0 0 1)

In [93]:
(defn digit-sum ^long [n data]
    (reduce + 0 (map-indexed (fn [idx d] (* (pattern (inc n) idx) d )) data)))

(defn calc-digit ^long [n data]
    (mod (Math/abs (digit-sum n data)) 10))

(calc-digit 0 data)
(defn phase [data]
    (map-indexed (fn [idx d] (calc-digit idx data)) data))
(phase data)


(4 8 2 2 6 1 5 8)

In [94]:
(defn fft [input steps]
   (first 
    (drop 
     steps 
     (iterate phase input))))



#'user/fft

In [95]:
(println (fft (parse "69317163492948606335995924319873") 100))
(println (fft (parse "12345678") 4))
(time (println  (fft (parse (day-input) ) 100)))



(5 2 4 3 2 1 3 3 2 9 ...)
(0 1 0 2 9 4 9 8)
(1 9 9 4 4 4 4 7 1 2 ...)
"Elapsed time: 19202.5042 msecs"


nil

In [13]:
(def offset (read-string (apply str  (take 7 (day-input)))))
offset
         

5971873

In [419]:
(defn fast-digit [input n]
    (mod (apply + (drop (dec n) input)) 10))


#'user/fast-digit

In [420]:
(fast-digit [1 2 3 4 5] 3)

2

In [78]:
(defn ffft-step [input]
 (map-indexed 
  (fn [idx digit] (fast-digit input (inc idx)))
  input))

(ffft-step [1 2 3 4 5 6 7 8])



Syntax error compiling at (REPL:3:19).
Unable to resolve symbol: fast-digit in this context
Execution error (IllegalStateException) at user/eval5780 (REPL:6).
Attempting to call unbound fn: #'user/ffft-step


class java.lang.IllegalStateException: 

In [422]:
(defn ffft [input steps]
   (first 
    (drop 
     steps 
     (iterate ffft-step input))))



#'user/ffft

In [427]:
(println (ffft (parse "69317163492948606335995924319873") 100 ))
(println (ffft (parse "12345678") 4))
(time (println  (ffft (parse (day-input)) 100)))



(7 1 7 7 9 4 8 8 4 9 ...)
(8 8 2 4 9 4 9 8)
(5 0 1 2 7 0 2 2 6 9 ...)
"Elapsed time: 3238.236323 msecs"


nil

In [412]:
(def input2 (apply vector (flatten (repeat 10000 (parse (day-input))))))
(time (println (take 8  (drop offset (ffft input2 100)))))
;;(count input2)


interrupted: 

In [447]:
(defn day16 [^long part input]
  (let [signal (mapv #(Character/digit (char %) 10) (s/trim input))]
    (case part
      1 (let [pattern (fn [n] (rest (cycle (sequence (comp (map (partial repeat n)) cat) [0 1 0 -1]))))
              pattern-m (fn [n] (mapv #(into [] (take n) (pattern %)) (range 1 (inc n))))
              last-digit (fn [^long x] (rem (Math/abs x) 10))
              dot (fn [v w] (last-digit (reduce + (mapv * v w))))
              mmul (fn [m v] (mapv (partial dot v) m))]
          (apply str (take 8 (nth (iterate (partial mmul (pattern-m (count signal))) signal) 100))))
      2 (let [phase (fn [^ints arr] (loop [n (- (count arr) 2)]
                                      (when (>= n 0)
                                        (aset arr n (rem (+ (aget arr n) (aget arr (inc n))) 10))
                                        (recur (dec n)))))
              offset (Integer/parseInt (subs input 0 7))]
          (let [m (- (* 10000 (count signal)) offset)
                left (rem m (count signal))
                arr (int-array (take m (concat (take-last left signal) (cycle signal))))]
            (dotimes [_ 100] (phase arr)) (apply str (take 8 arr)))))))

#'user/day16

In [1]:
(time (day16 2 (day-input)))

Syntax error compiling at (REPL:1:7).
Unable to resolve symbol: day16 in this context


class clojure.lang.Compiler$CompilerException: 

In [440]:
(mapv #(Character/digit (char %) 10) (s/trim (day-input)))

[5 9 7 1 8 7 3 0 6 0 ...]